In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


class TeacherModel(nn.Module):
    def __init__(self):
        super(TeacherModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.fc3 = nn.Linear(1200, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class StudentModel(nn.Module):
    def __init__(self):
        super(StudentModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 800)
        self.fc2 = nn.Linear(800, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 485kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.43MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.89MB/s]


In [2]:
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher = F.softmax(teacher_logits / temperature, dim=1)
    soft_student = F.log_softmax(student_logits / temperature, dim=1)
    kl_div = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (temperature ** 2)
    ce_loss = F.cross_entropy(student_logits, labels)
    return alpha * kl_div + (1 - alpha) * ce_loss

teacher = TeacherModel()
optimizer = optim.Adam(teacher.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
def train_teacher(model, train_loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f"Teacher Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train_teacher(teacher, train_loader, optimizer, criterion)

student = StudentModel()
optimizer = optim.Adam(student.parameters(), lr=0.001)

def train_student(student, teacher, train_loader, optimizer, temperature=5.0, alpha=0.7, epochs=5):
    student.train()
    teacher.eval()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            student_logits = student(data)
            with torch.no_grad():
                teacher_logits = teacher(data)
            loss = distillation_loss(student_logits, teacher_logits, target, temperature, alpha)
            loss.backward()
            optimizer.step()
        print(f"Student Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train_student(student, teacher, train_loader, optimizer)
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

print("Teacher Model Accuracy:")
teacher_accuracy = evaluate(teacher, test_loader)

print("Student Model Accuracy:")
student_accuracy = evaluate(student, test_loader)


Teacher Epoch 1/5, Loss: 0.5995293855667114
Teacher Epoch 2/5, Loss: 0.28969675302505493
Teacher Epoch 3/5, Loss: 0.11444567888975143
Teacher Epoch 4/5, Loss: 0.04127619415521622
Teacher Epoch 5/5, Loss: 0.005713932681828737
Student Epoch 1/5, Loss: 0.6931514739990234
Student Epoch 2/5, Loss: 0.35999104380607605
Student Epoch 3/5, Loss: 0.2561858892440796
Student Epoch 4/5, Loss: 0.2926703989505768
Student Epoch 5/5, Loss: 0.15706883370876312
Teacher Model Accuracy:
Accuracy: 97.25%
Student Model Accuracy:
Accuracy: 97.09%


In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

teacher_params = count_parameters(teacher)
student_params = count_parameters(student)

print(f"Teacher Model Parameters: {teacher_params}")
print(f"Student Model Parameters: {student_params}")

print(f"Teacher Accuracy: {teacher_accuracy:.2f}%, Parameters: {teacher_params}")
print(f"Student Accuracy: {student_accuracy:.2f}%, Parameters: {student_params}")


Teacher Model Parameters: 2395210
Student Model Parameters: 636010
Teacher Accuracy: 97.25%, Parameters: 2395210
Student Accuracy: 97.09%, Parameters: 636010
